In [ ]:
# This script implements a simple trading strategy based on a
# 200-day moving average and a 2-period RSI.
# It can now backtest a portfolio of up to 5 tickers simultaneously.
# Install the required libraries if you haven't already:
# pip install yfinance pandas matplotlib numpy

import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import itertools

def calculate_rsi(data, period):
    """Calculates the Relative Strength Index (RSI)."""
    delta = data.diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def main():
    """Main function to run the trading strategy on a portfolio."""
    # --- Configuration ---
    # User can provide up to 5 ticker symbols.
    ticker_symbols = ["BBCA.JK", "BMRI.JK", "TLKM.JK"]
    if not 1 <= len(ticker_symbols) <= 5:
        print("Error: Please provide between 1 and 5 ticker symbols.")
        return

    start_date = "2010-01-01"
    end_date = pd.Timestamp.today().strftime('%Y-%m-%d')
    initial_capital = 10000.0
    number_of_tickers = len(ticker_symbols)

    # Calculate capital per ticker
    capital_per_ticker = initial_capital / len(ticker_symbols)

    # --- Data Acquisition & Indicator Calculation for all tickers ---
    print(f"Downloading daily data for {ticker_symbols} from {start_date} to {end_date}...")

    portfolio_data = {}
    try:
        for ticker in ticker_symbols:
            df = yf.download(ticker, start=start_date, end=end_date)
            if df.empty:
                print(f"Warning: No data downloaded for {ticker}. Skipping.")
                continue

            df['SMA_200'] = df['Close'].rolling(window=200).mean()
            df['RSI'] = calculate_rsi(df['Close'], period=2)
            portfolio_data[ticker] = df

    except Exception as e:
        print(f"An error occurred while downloading data: {e}")
        return

    if not portfolio_data:
        print("Error: No valid data downloaded for any ticker.")
        return

    print("Data downloaded and indicators calculated successfully.")

    # --- Backtesting the Strategy on the Portfolio ---

    # Get all unique dates from all tickers' indices and convert to Series
    all_dates = pd.concat([pd.Series(df.index) for df in portfolio_data.values()]).drop_duplicates().sort_values()

    # Initialize portfolio variables
    portfolio_equity = [initial_capital]

    # Dictionary to track per-ticker state
    portfolio_state = {ticker: {'in_position': False, 'shares': 0, 'cash': capital_per_ticker, 'entry_dates': [], 'exit_dates': []} for ticker in portfolio_data.keys()}

    # Find the common start date for backtesting
    # The backtesting should start after the 200-day SMA period for all tickers.
    min_start_date = max(df.index[200] for df in portfolio_data.values())

    # Iterate through the common dates
    start_index = all_dates[all_dates >= min_start_date].index[0]

    for i in range(start_index, len(all_dates) - 1):
        current_date = all_dates.iloc[i]

        current_portfolio_value = 0

        for ticker, df in portfolio_data.items():
            if current_date in df.index:
                state = portfolio_state[ticker]
                current_day = df.loc[current_date]

                # Check for buy signal
                if (not state['in_position'] and
                    not pd.isna(current_day['SMA_200']).item() and
                    not pd.isna(current_day['RSI']).item() and
                    current_day['Close'].item() > current_day['SMA_200'].item() and
                    current_day['RSI'].item() < 10):

                    shares_to_buy = state['cash'] // current_day['Close'].item()
                    if shares_to_buy > 0:
                        state['shares'] += shares_to_buy
                        state['cash'] -= shares_to_buy * current_day['Close'].item()
                        state['in_position'] = True
                        state['entry_dates'].append(current_day.name)
                        print(f"BUY Signal on {ticker} on {current_day.name.date()}: Executing buy at close (${current_day['Close'].item():.2f})")

                # Check for sell signal
                if state['in_position'] and i > 0 and all_dates.iloc[i-1] in df.index:
                    previous_day_idx = df.index.get_loc(all_dates.iloc[i-1])
                    previous_day = df.iloc[previous_day_idx]
                    if current_day['Close'].item() > previous_day['High'].item():
                        state['cash'] += state['shares'] * current_day['Close'].item()
                        state['shares'] = 0
                        state['in_position'] = False
                        state['exit_dates'].append(current_day.name)
                        print(f"SELL Signal on {ticker} on {current_day.name.date()}: Executing sell at close (${current_day['Close'].item():.2f})")

                # Update individual ticker value
                current_portfolio_value += state['cash'] + state['shares'] * current_day['Close'].item()

            else:
                # If a ticker has no data for the current day, its value is its last known value
                state = portfolio_state[ticker]
                last_known_close = df['Close'].iloc[-1].item() if not df.empty else 0
                current_portfolio_value += state['cash'] + state['shares'] * last_known_close

        portfolio_equity.append(current_portfolio_value)

    # The rest of the code for plotting and final results remains the same.

    # --- Plotting the Results ---
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)
    fig.suptitle(f'Portfolio Trading Strategy Backtest', fontsize=16)

    # Plot 1: Individual Ticker Prices and SMA
    for ticker, df in portfolio_data.items():
        ax1.plot(df.index, df['Close'], label=f'{ticker} Closing Price', alpha=0.8)
        ax1.plot(df.index, df['SMA_200'], linestyle='--', label=f'{ticker} SMA_200')

    ax1.set_title(f'Stock Prices and 200-Day SMA for Portfolio')
    ax1.set_ylabel('Price ($)')
    ax1.grid(True, which='both', linestyle='--', linewidth=0.5)
    ax1.legend()

    # Add buy and sell markers for each ticker
    marker_style = itertools.cycle(['o', '^', 'v', 's', 'p'])
    marker_colors = itertools.cycle(['green', 'red', 'purple', 'blue', 'orange'])

    for ticker, state in portfolio_state.items():
        marker = next(marker_style)
        color = next(marker_colors)
        df = portfolio_data[ticker]

        buy_prices = [df.loc[date]['Close'] for date in state['entry_dates']]
        sell_prices = [df.loc[date]['Close'] for date in state['exit_dates']]

        ax1.scatter(state['entry_dates'], buy_prices, marker=marker, color='green', s=100, label=f'{ticker} Buy Signal')
        ax1.scatter(state['exit_dates'], sell_prices, marker=marker, color='red', s=100, label=f'{ticker} Sell Signal')

    ax1.text(1.0, 0.0, 'backtested by Handiko', transform=ax1.transAxes, fontsize=12, color='darkgray', ha='right', va='bottom', fontweight='bold')

    # Plot 2: Equity Growth on a logarithmic scale
    equity_df = pd.Series(portfolio_equity, index=all_dates[start_index:])
    ax2.plot(equity_df.index, equity_df.values, label='Portfolio Equity Growth', color='purple')
    ax2.axhline(y=initial_capital, color='gray', linestyle='--', label='Initial Capital')
    ax2.set_ylabel('Equity ($)')
    ax2.set_yscale('log')
    ax2.grid(True, which='both', linestyle='--', linewidth=0.5)
    ax2.legend(loc='upper left')

    ax2.set_title('Portfolio Equity Growth (Log Scale)')
    ax2.set_xlabel('Date')

    ax2.text(1.0, 0.0, 'backtested by Handiko', transform=ax2.transAxes, fontsize=12, color='darkgray', ha='right', va='bottom', fontweight='bold')

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])

    plot_filename = f"portfolio_trading_strategy_plot_{number_of_tickers}_tickers.png"
    plt.savefig(plot_filename)
    print(f"\nPlot saved to {plot_filename}")

    # --- Final Results ---
    final_equity = portfolio_equity[-1]
    total_return = (final_equity - initial_capital) / initial_capital * 100
    print(f"\n--- Final Results ---")
    print(f"Initial Capital: ${initial_capital:.2f}")
    print(f"Final Portfolio Equity: ${final_equity:.2f}")
    print(f"Total Portfolio Return: {total_return:.2f}%")

if __name__ == "__main__":
    main()